<center>

**CRIPTOGRAFÍA DE LLAVE PÚBLICA**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

# **🧰El Gamal🧷**

<p align="center">
    <img src="https://files.codingninjas.in/article_images/hello-i-am-elgamal-cryptosystem-0-1670342463.webp"width="400">
</p>

<div align="justify">

**Parámetros del sistema**

- **Primo seguro**  
   Elegimos un número primo grande  
   $$
     p\in\mathbb{P},\quad \text{con cardinalidad normal de }256\text{, }512\text{ o }1024\text{ bits.}
   $$

- **Generador (raíz primitiva)**  
   Seleccionamos  
   $$
     g\in(\mathbb{Z}/p)^\times
   $$  
   tal que su orden sea exactamente $p-1$, es decir, $g$ genera el grupo multiplicativo.

- **Clave privada**  
   Bob elige un exponente secreto  
   $$
     a\;\in\;\{1,2,\dots,p-2\}
   $$

- **Clave pública**  
   Calcula  
   $$
     h \;=\; g^a \bmod p.
   $$  
   La clave pública es el triplete  
   $$
     (p,\;g,\;h),
   $$  
   mientras que $a$ se mantiene en secreto.

---

**Representación de mensajes**

- Para poder cifrar textos arbitrarios (UTF-8), primero convertimos la cadena de caracteres en un entero:

  $$\text{bytes} = \mathrm{UTF8}(\text{texto}), \quad m = \mathrm{bytes\_to\_long}(\text{bytes}).$$
    
- Tras el descifrado, convertimos de nuevo el entero a bytes y luego a texto:

  $$
    \text{bytes} = \mathrm{long\_to\_bytes}(m),
    \quad
    \text{texto} = \mathrm{UTF8\_decode}(\text{bytes}).
  $$

---

**Cifrado**

Para cifrar un mensaje representado por el entero $m$:

  -  **Nonce aleatorio**  
   Elegimos $$k$$ uniformemente en $$\{1,\dots,p-2\}$$ con  
   $$
     \gcd(k,p-1)=1.
   $$

  - **Cálculo de los componentes**  
   $$
     c_1 = g^k \bmod p,
     \qquad
     c_2 = m \;\cdot\; h^k \bmod p.
   $$

  - **Salida del ciphertext**  
   Se envía el par  
   $$
     (\,c_1,\;c_2).
   $$

---

**Descifrado**

Dado el ciphertext $$(c_1,c_2)$$ y conociendo la clave privada $a$:

-  **Factor compartido**  
   $$
     s = c_1^a \bmod p
     \quad(\text{equivale a }g^{k\,a}=h^k).
   $$

-  **Inverso modular**  
   $$
     s^{-1} \;=\; \bigl(c_1^a\bigr)^{-1}\bmod p.
   $$

-  **Recuperación de $$m$$**  
   $$
     m = c_2 \;\cdot\; s^{-1}\bmod p.
   $$

-  **Reconversión a texto**  
   $$
     \text{bytes} = \mathrm{long\_to\_bytes}(m),
     \quad
     \text{texto} = \mathrm{UTF8\_decode}(\text{bytes}).
   $$

---

**Propiedades de seguridad**

- La seguridad se basa en la dificultad del problema del logaritmo discreto en $(\mathbb{Z}/p)^\times$: dado $(g,p,h)$ es conjeturalmente intratable recuperar $a$ de $h=g^a\pmod p$.

- El nonce $k$ debe ser único y aleatorio por cada cifrado para evitar ataques de reutilización de clave.

- Se recomienda usar $p$ de al menos 2048 bits en entornos reales.

---

**Instrucciones de manejo**

- **Bits de p**: elección del tamaño de $p$ en bits.  

- **Generar claves**: construye $p,g,h,a$.  

- **Mensaje**: texto libre que se convierte a bytes e entero.

- **Cifrar → Descifrar**: ejecuta el protocolo completo y muestra  
  - $c_1, c_2, k$  
  - $s = c_1^a$ y su inverso  
  - Bytes y texto recuperados  


</div>

**📬Instalar paquetes📥**

In [7]:

!pip install pycryptodome ipywidgets

**📥Importaciones📦**

In [8]:

import random, math
import ipywidgets as wd
from IPython.display import display, HTML, clear_output
from Crypto.PublicKey import ElGamal
from Crypto import Random
from Crypto.Util.number import bytes_to_long, long_to_bytes, inverse

**👨‍💻Implementación👩‍💻**

In [9]:

def gen_elgamal_keys(bits):
    key = ElGamal.generate(bits, Random.get_random_bytes)
    pub = key.publickey()
    # convertir a int nativo
    return {
        'p': int(key.p),
        'g': int(key.g),
        'y': int(pub.y),
        'x': int(key.x)
    }

def elgamal_encrypt(params, message_bytes):
    p, g, y = params['p'], params['g'], params['y']
    m = bytes_to_long(message_bytes)
    # escoge k con gcd(k, p-1)=1
    while True:
        k = random.randrange(1, p-1)
        if math.gcd(k, p-1)==1:
            break
    c1 = pow(g, k, p)
    c2 = (m * pow(y, k, p)) % p
    return c1, c2, k

def elgamal_decrypt(params, c1, c2):
    p, x = params['p'], params['x']
    s = pow(c1, x, p)
    si = inverse(s, p)
    m = (c2 * si) % p
    return long_to_bytes(m)

**👨‍💻Implementación👩‍💻**

In [10]:

bits_widget = wd.BoundedIntText(value=256, min=64, max=2048, step=64,
                                description='Bits p:')
btn_gen     = wd.Button(description='Generar claves', button_style='info', layout=wd.Layout(width='200px'))
keys_html   = wd.HTML(value="<i>Pulse Generar claves</i>")

msg_widget  = wd.Text(value='¡Hola, mundo!', description='Mensaje:')
btn_enc     = wd.Button(description='Cifrar → Descifrar', button_style='success', layout=wd.Layout(width='200px'))
result_html = wd.HTML(value="<i>Aquí aparecerán los resultados</i>")

_elgamal_params = {}

**👨‍💻Implementación👩‍💻**

In [11]:

def on_generate(_):
    clear_output(wait=True)
    params = gen_elgamal_keys(bits_widget.value)
    _elgamal_params.update(params)
    keys_html.value = f"""
    <h3 style="color:#336699">Claves ElGamal Generadas</h3>
    <b>p</b> = {params['p']}<br>
    <b>g</b> = {params['g']}<br>
    <b>y</b> (pública) = {params['y']}<br>
    <b>x</b> (privada) = {params['x']}<br>
    <hr>
    <i>Ahora introduzca un mensaje y pulse "Cifrar → Descifrar".</i>
    """

def on_encrypt(_):
    clear_output(wait=True)
    if 'p' not in _elgamal_params:
        result_html.value = "<span style='color:red;'>Primero genere las claves.</span>"
        return
    texto = msg_widget.value
    mensaje = texto.encode('utf-8')
    c1, c2, k = elgamal_encrypt(_elgamal_params, mensaje)
    pt = elgamal_decrypt(_elgamal_params, c1, c2)
    try:
        pt_text = pt.decode('utf-8')
    except:
        pt_text = str(pt)
    result_html.value = f"""
    <h3 style="color:#336699">Resultado ElGamal</h3>
    <b>Mensaje original:</b> <code>{texto}</code><br>
    <b>Bytes:</b> <code>{mensaje}</code><hr>
    <b>Cifrado:</b><br>
    c1 = {c1}<br>
    c2 = {c2}<br>
    <b>Nonce k:</b> {k}<hr>
    <b>Descifrado:</b><br>
    Bytes → <code>{pt}</code><br>
    Texto → <code>{pt_text}</code>
    """

btn_gen.on_click(on_generate)
btn_enc.on_click(on_encrypt)

**👨‍💻Implementación👩‍💻**

In [12]:

ui = wd.VBox([
    wd.HTML("<h2 style='color:#336699'>🔐 ElGamal</h2>"),
    wd.HBox([bits_widget, btn_gen]),
    keys_html,
    wd.HTML("<hr>"),
    wd.HBox([msg_widget, btn_enc]),
    result_html
])

display(ui)